In [18]:
from pathlib import Path
import socket
import requests
import requests.packages.urllib3.util.connection as urllib3_cn
import TreeAnnotations 

In [19]:
# Some globals

neon_image_product_code = "DP3.30010.001"
site = "JORN"
year_month = "2019-08"

#keeping this low until all the moving parts work (development phase)
downloadCount = 5;
defaultPhotoDropPath = Path(".")
apiBase = 'https://data.neonscience.org/api/v0/'


In [20]:
def allowed_gai_family():
    return socket.AF_INET

urllib3_cn.allowed_gai_family = allowed_gai_family

In [21]:
# Get information about the image data set
def getProductInfo(neon_image_product_code):
    product_info_endpoint = f'products/{neon_image_product_code}'
    product_details = requests.get(apiBase+product_info_endpoint)
    #print(product_details.content)
    return product_details.content

# Returns a list of dictionaries with the photo name and download URL
def getListOfSiteFilesForYear(dataEndpointWithParams):
    responseWithListOfSiteFiles = requests.get(apiBase + dataEndpointWithParams)
    return responseWithListOfSiteFiles.json()['data']['files']

# downloads a single photo to the photoDropPath
def download_image(photoName, photoURL, photoDropPath = defaultPhotoDropPath):    
    photoRequest = requests.get(photoURL, allow_redirects=True)
    with open(photoDropPath / photoName, 'wb') as f:
       f.write(photoRequest.content)

# downloads photos from the list of dictionaries returned from getListOfSiteFilesForYear
def download_images(image_dictionary, photoDropPath = defaultPhotoDropPath):
    for entry in image_dictionary:
        download_image(entry['name'], entry['url'], photoDropPath)

In [22]:
dataEndpointWithParams = f'data/{neon_image_product_code}/{site}/{year_month}?package=basic'
image_list_dictionary = getListOfSiteFilesForYear(dataEndpointWithParams)

image_list_dictionary = image_list_dictionary[:downloadCount]
download_images(image_list_dictionary)

In [17]:
TreeAnnotations.downloadAnnotation(site, defaultPhotoDropPath/ f"{site}.csv")